In [1]:
!python --version

Python 3.6.9


In [2]:
!pip install sentence-transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=c30b6c812ac850a2171d7537d2e27b95274be02e1bfa6e1cbcd9c204f81bacaa
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=aeadc7d0004e60d22ee246a137ac13ae5da1ea23b32fc0ca0a63304329a092e7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [3]:
import pandas as pd
import pickle as pkl

df = pd.read_csv("/content/drive/My Drive/COVID Press/covid_press_QA.csv")
df.head()

,questions,answers,time,transcript_link,date,header,main_page_counter
0,Governor Cuomo. You mentioned that New York or...,It’s always the same. You start reopening that...,33:29,https://www.rev.com/transcript-editor/shared/_...,"May 12, 2020",Andrew Cuomo New York COVID-19 Press Conferenc...,0
1,"When you talk about closing the valve, say we ...","Well, if you’re calibrating the reopening and ...",34:09,https://www.rev.com/transcript-editor/shared/_...,"May 12, 2020",Andrew Cuomo New York COVID-19 Press Conferenc...,0
2,"Governor, one major Southern Tier institution ...",We’ll work with any employer on how to adopt s...,35:28,https://www.rev.com/transcript-editor/shared/_...,"May 12, 2020",Andrew Cuomo New York COVID-19 Press Conferenc...,0
3,"Governor, you talked about staying smart and w...",Yeah. That’s going to be up to the local healt...,36:14,https://www.rev.com/transcript-editor/shared/_...,"May 12, 2020",Andrew Cuomo New York COVID-19 Press Conferenc...,0
4,Thank you. Is New York setting up a system in ...,We live in fear of a second wave. If you see t...,36:59,https://www.rev.com/transcript-editor/shared/_...,"May 12, 2020",Andrew Cuomo New York COVID-19 Press Conferenc...,0


In [0]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models, losses
import scipy.spatial
import pickle as pkl
word_embedding_model = models.BERT("/content/drive/My Drive/COVID Press/BertCovid")

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
questions = list(df["questions"])
corpus_embeddings = model.encode(questions)
import pickle as pkl
with open("/content/drive/My Drive/COVID Press/corpus_embeddings.pkl" , "wb") as file_:
  pkl.dump(corpus_embeddings,file_)

In [0]:
with open("/content/drive/My Drive/COVID Press/corpus_embeddings.pkl" , "rb") as file_:
  corpus_embeddings = pkl.load(file_)

In [50]:
# Query sentences:
queries = ['how the virus was started',
           'when would life return to normal',
           'how many tests would we able to perform',      
           'how would you support the unemployed',
           'do we have enough ventilators',
          ]
query_embeddings = model.encode(queries,show_progress_bar=True)

# Find the closest 2 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 2
print("\nTop 2 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("question:   ", questions[idx].strip(), "\n" )
        row_dict = df.loc[df.index== idx].to_dict()
        print("answers:  " , row_dict["answers"][idx], "\n")
        print("header:  " , row_dict["header"][idx] , "\n")
        print("date:  " , row_dict["date"][idx] , "\n")
        print("time:  " , row_dict["time"][idx] , "\n")
        print("transcript_link:  " , row_dict["transcript_link"][idx] , "\n")
        print("-------------------------------------------")

Batches: 100%|██████████| 1/1 [00:00<00:00, 33.36it/s]



Top 2 most similar sentences in corpus:


==========================Query==============================
=== how the virus was started =====
Score:    (Score: 0.7392) 

question:    The investigation into whether the virus escaped from this lab in Wuhan, how active is that, and when do you expect- 

answers:   Well, we’re looking at it. A lot of people are looking at it. It seems to make sense. They talk about a certain kind of bat, but that bat wasn’t in that area, if you can believe this. That’s what they’re down to now is bats. But that bat is not in that area. That bat wasn’t sold at that wet zone. It wasn’t sold there. That bat is 40 miles away, so a lot of strange things are happening, but there is a lot of investigation going on, and we’re going to find out. All I can say is wherever it came from… It came from China. In whatever form, 184 countries now are suffering because of it, and it’s too bad, isn’t it? And it could have been solved very easily when it was just starting.It 